In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("data/raw/sentiment.csv")
df.head()

,Article,m_sentiment,f_sentiment,m_emotion,f_emotion,m_normalized,f_normalized
0,f : there are many wonderful places for shoppi...,0.500000,0.534091,Satisfaction,Joy,-1.250000,-1.164773
1,f : what is it frank ? you look upset . m : hi...,0.106944,0.106944,Satisfaction,Satisfaction,-2.232639,-2.232639
2,"f : good morning , i was just walking by and s...",0.285714,0.492857,Satisfaction,Satisfaction,-1.785714,-1.267857
3,"m : i 'm afraid you 've got a temperature , ma...",-0.600000,-0.333333,Sadness,Disappointment,-4.000000,-3.333333
4,f : good morning . m : good morning . f : mrs ...,0.700000,0.700000,Joy,Joy,-0.750000,-0.750000


In [3]:
model_df_dict = {"sentences": [], "label": []}


def sentiment_to_label(sentiment):
    if sentiment < -0.5:
        return 0
    elif sentiment > 0.5:
        return 2
    return 1


for i in range(df.shape[0]):
    df_i = df.iloc[i]

    conversation = df_i["Article"]

    # m : {sentence} f :
    m = " | ".join(re.findall(r"(?<=m : ).*?(?=f : )", conversation))
    m_label = df_i.m_sentiment

    # f : {sentence} m :
    f = " | ".join(re.findall(r"(?<=f : ).*?(?=m : )", conversation))
    f_label = df_i.f_sentiment

    if m is not None and len(m) > 0:
        model_df_dict["sentences"].append(m)
        model_df_dict["label"].append(sentiment_to_label(m_label))

    if f is not None and len(f) > 0:
        model_df_dict["sentences"].append(f)
        model_df_dict["label"].append(sentiment_to_label(f_label))

In [4]:
model_df = pd.DataFrame.from_dict(model_df_dict).dropna()
model_df.head()

,sentences,label
0,so do i. i like here because i can watch footb...,1
1,there are many wonderful places for shopping i...,2
2,"hi nancy , i work very hard , i get results . ...",1
3,what is it frank ? you look upset . | frank i...,1
4,that is right .,1


In [5]:
train, test = (
    model_df[: int(0.8 * model_df.shape[0])],
    model_df[int(0.8 * model_df.shape[0]) :],
)

In [6]:
train.to_csv("data/processed/train.csv", index=False)
test.to_csv("data/processed/test.csv", index=False)